# NOPT042 Constraint programming: Tutorial 4 – Search strategies

In [12]:
%load_ext ipicat

<IPython.core.display.Javascript object>

Picat version 3.2#8


### From last week:

* Solution to the Coin grid problem.
* Best model and solver for the problem? MIP, naturally expressed as an integer program
* Unsatisfiable instances - LP works well.
* For sparse solution sets heuristic approaches may be slow.

## Example: N-queens

Place $n$ queens on an $n\times n$ board so that none attack another. How to choose the decision variables?

* How large is the search space?
* Can we use symmetry breaking?
* Consider the _dual_ model.

In [2]:
!time picat queens/queens-primal.pi 8

.....Q..
...Q....
.Q......
.......Q
....Q...
......Q.
Q.......
..Q.....

real	0m0.014s
user	0m0.007s
sys	0m0.007s


In [3]:
!cat queens/queens-primal.pi

% n-queens, primal model
import sat.

main([N]) =>
    N := to_int(N),
    queens(N, Q),
    solve(Q),
    if N <= 32 then
        output(Q)
    end.


queens(N, Q) =>
    Q = new_array(N),
    Q :: 1..N,
    all_different(Q),
    all_different([$Q[I] - I : I in 1..N]),
    all_different([$Q[I] + I : I in 1..N]).


output(Q) =>
    N = Q.length,
    foreach(I in 1..N)
        foreach (J in 1..N)
            if Q[I] = J then
                print("Q")
            else
                print(".")
            end
        end,
        print("\n")
    end.


In [5]:
!time picat queens/queens-dual.pi 64

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --lp -o __tmp.sol __tmp.lp
Reading problem data from '__tmp.lp'...
8575 rows, 4097 columns, 28672 non-zeros
4097 integer variables, 4096 of which are binary
16775 lines were read
GLPK Integer Optimizer 5.0
8575 rows, 4097 columns, 28672 non-zeros
4097 integer variables, 4096 of which are binary
Preprocessing...
378 hidden packing inequaliti(es) were detected
379 rows, 4096 columns, 20476 non-zeros
4096 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 379
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
379 rows, 4096 columns, 20476 non-zeros
      0: obj =   0.000000000e+00 inf =   3.150e+02 (5)
    434: obj =   0.000000000e+00 inf =   8.882e-16 (0) 2
OPTIMAL LP SOLUTION FOUND
Integer optimization begins...
Long-step dual simplex will be used
+

In [7]:
!cat queens/queens-dual.pi

% n-queens, dual model
import mip.

main([N]) =>
    N := to_int(N),
    queens(N, Board),
    solve(Board),
    if N <= 32 then
        output(Q)
    end.


queens(N, Board) =>
    Board = new_array(N, N),
    Board :: 0..1,
    
    sum([Board[I, J] : I in 1..N, J in 1..N]) #= N,
    
    % rows
    foreach(I in 1..N)
        sum([Board[I, J] : J in 1..N]) #<= 1
    end,
    
    % cols
    foreach(J in 1..N)
        sum([Board[I, J] : I in 1..N]) #<= 1
    end,

    % diags
    foreach(K in 1-N..N-1)
        sum([Board[I,J] : I in 1..N, J in 1..N, I-J = K ]) #<= 1
    end,
    foreach(K in 2..2*N)
        sum([Board[I,J] : I in 1..N, J in 1..N, I+J = K ]) #<= 1
    end.
    

output(Board) =>
    N = Board.length,
    foreach(I in 1..N)
        foreach (J in 1..N)
            if Board[I, J] = 1 then
                print("Q")
            else
                print(".")
            end
        end,
        print("\n")
    end.


Sometimes it is best to model the problem in both ways and add _channelling constraints_. (Here it does not help.)

In [8]:
!time picat queens/queens-channeling.pi 128


real	0m6.805s
user	0m6.752s
sys	0m0.032s


In [10]:
!cat queens/queens-channeling.pi

% n-queens, primal model
import sat.

main([N]) =>
    N := to_int(N),
    queens(N, Q, Board),
    solve(Q ++ Board),
    if N <= 32 then   
        output(Q)
    end.


queens(N, Q, Board) =>    
    % primal
    Q = new_array(N),
    Q :: 1..N,
    all_different(Q),
    all_different([$Q[I] - I : I in 1..N]),
    all_different([$Q[I] + I : I in 1..N]),

    % dual
    Board = new_array(N, N),
    Board :: 0..1,    
    sum([Board[I, J] : I in 1..N, J in 1..N]) #= N,
    foreach(I in 1..N)
        sum([Board[I, J] : J in 1..N]) #<= 1
    end,
    foreach(J in 1..N)
        sum([Board[I, J] : I in 1..N]) #<= 1
    end,
    foreach(K in 1-N..N-1)
        sum([Board[I, J] : I in 1..N, J in 1..N, I - J = K ]) #<= 1
    end,
    foreach(K in 2..2*N)
        sum([Board[I, J] : I in 1..N, J in 1..N, I + J = K ]) #<= 1
    end,
    
    % channeling
    foreach(I in 1..N, J in 1..N)
        (Board[I,J] #= 1) #<=> (Q[I] #= J)
    end.


output(Q) =>
    N = Q.length,
    foreach(I in 1..N)
  

Can the models be improved using symmetry breaking?

## Search strategies
And other solver options: see [Picat guide](http://picat-lang.org/download/picat_guide.pdf) (Section 12.6) and the [book](http://picat-lang.org/picatbook2015/constraint_solving_and_planning_with_picat.pdf) (Section 3.5)

In [13]:
%%picat -n queens
import cp. %try sat, try also mip with the other model

queens(N, Q) =>
    Q = new_array(N),
    Q :: 1..N,
    all_different(Q),
    all_different([$Q[I] - I : I in 1..N]),
    all_different([$Q[I] + I : I in 1..N]).

The predicate `time2` also outputs the number of backtracks during the search - a good measure of complexity.

In [14]:
%%picat
main =>
    N = 32,
    queens(N, Q),
    time2(solve(Q)).


CPU time 25.24 seconds. Backtracks: 11461548




Which search strategy could work well for our model?

Here's how we can test multiple search strategies (code adapted from [the book](https://picat-lang.org/picatbook2015/constraint_solving_and_planning_with_picat.pdf)):

In [15]:
%%picat

% Variable selection
selection(VarSels) =>
    VarSels = [backward,constr,degree,ff,ffc,ffd,forward,inout,leftmost,max,min,up].
    
% Value selection
choice(ValSels) =>
    ValSels = [down,reverse_split,split,up,updown].

main =>
    selection(VarSels),
    choice(ValSels),
    Timeout = 1000, % Timeout in milliseconds
    %Timeout = 10000, % Timeout in milliseconds
    Ns = [64, 128, 256],
    
    foreach (N in Ns, VarSel in VarSels, ValSel in ValSels)
        queens(N,Q),
        time2(time_out(solve([VarSel,ValSel], Q),Timeout,Status)),
        println([N,VarSel,ValSel,Status])
    end.


CPU time 0.998 seconds. Backtracks: 449995

[64,backward,down,time_out]

CPU time 0.998 seconds. Backtracks: 0

[64,backward,reverse_split,time_out]

CPU time 0.998 seconds. Backtracks: 0

[64,backward,split,time_out]

CPU time 0.998 seconds. Backtracks: 286461

[64,backward,up,time_out]

CPU time 0.998 seconds. Backtracks: 718810

[64,backward,updown,time_out]

CPU time 0.997 seconds. Backtracks: 440335

[64,constr,down,time_out]

CPU time 0.996 seconds. Backtracks: 0

[64,constr,reverse_split,time_out]

CPU time 0.998 seconds. Backtracks: 0

[64,constr,split,time_out]

CPU time 0.999 seconds. Backtracks: 275173

[64,constr,up,time_out]

CPU time 0.998 seconds. Backtracks: 669809

[64,constr,updown,time_out]

CPU time 0.998 seconds. Backtracks: 443592

[64,degree,down,time_out]

CPU time 0.998 seconds. Backtracks: 0

[64,degree,reverse_split,time_out]

CPU time 0.998 seconds. Backtracks: 0

[64,degree,split,time_out]

CPU time 0.998 seconds. Backtracks: 272322

[64,degree,up,time_out

## Exercises

### Exercise: Magic square
Arrange numbers $1,2,\dots,n^2$ in a square such that every row, every column, and the two main diagonals all sum to the same quantity. 

* Try to find the best model, solver and search strategy.
* How many magic squares are there for a given $n$? 
* Allow also for a partially filled instance. 

### Exercise: Minesweeper
Identify the positions of all mines in a given board. Try the following instance (from [the book](http://picat-lang.org/picatbook2015/constraint_solving_and_planning_with_picat.pdf)):
```
Instance = {
    {_,_,2,_,3,_},
    {2,_,_,_,_,_},
    {_,_,2,4,_,3},
    {1,_,3,4,_,_},
    {_,_,_,_,_,3},
    {_,3,_,3,_,_}
}.
```

### Exercise: Graph-coloring
1. Write a program that solves the (directed) graph 3-coloring problem with a given number of colors and a given graph. The graph is given by a list of edges, each edge is a 2-element list. We assume that vertices of the graph are $1,\dots,n$ where $n$ is the maximum number appearing in the list. 
2. Generalize your program to graph $k$-coloring where $k$ is a positive integer given on the input.
3. Modify your program to accept the incidence matrix (a 2D array) instead of the list of edges.
4. Add the flag `-n` to output the minimum number of colors (the chromatic number) of a given graph.
For example:
```
picat graph-coloring.pi [[1,2],[2,3],[3,4],[4,1]]
picat graph-coloring.pi [[1,2],[2,3],[3,1]] 4
picat graph-coloring.pi "{{0,1,1},{1,0,1},{1,1,0}}" 4
picat graph-coloring.pi -n [[1,2],[2,3],[3,4],[4,1]]
```

# Homework: knapsack

There are two common versions of the problem: the general __knapsack__ problem:

> Given a set of items, each with a weight and a value, determine __how many of each item__ to include in a collection so that the total weight is less than or equal to a given limit and the total value is as large as possible.

And the __0-1 knapsack__ problem:


> Given a set of items, each with a weight and a value, determine __which items__ to include in a collection so that the total weight is less than or equal to a given limit and the total value is as large as possible.

(In a general knapsack problem, we can take any number of each item, in the 0-1 version we can take at most one of each.)

#### Example of an instance:

A thief breaks into a department store (general knapsack) or into a home (0-1 knapsack). They can carry 23kg. Which items (and how many of each, in the general version) should they take to maximize profit? There are the following items:

* a TV (weighs 15kg, costs \$500),
* a desktop computer (weighs 11kg, costs \$350)
* a laptop (weighs 5kg, costs \$230), 
* a tablet (weighs 1kg, costs \$115),
* an antique vase (weighs 7kg, costs \$180), 
* a bottle of whisky (weighs 3kg, costs \$75), and 
* a leather jacket (weighs 4kg, costs \$125).

This instance is given in the file `data.pi`.

Your goal is to a program for both the problems. The models accept an optional flag "-01" to denote the 0-1 version, and a filename of a data file including the instance. The output should contain the optimal value, and some reasonable representation of the chosen items. (The autograder will only check the presence of the optimum value.)

Running
```
picat knapsack.pi data.pi
```
should output the optimal total value of 2645 and the chosen items `23 of tablet` in some reasonable format. Running 

```
picat knapsack.pi -01 data.pi
```
should output the optimal total value of 845 and the chosen items `[tv,laptop,tablet]` in some reasonable format.


Use the solver `cp` (even though `mip` would be better here) and try to find the best model and the best search strategy. You will need to generate larger instances. You can do it in Picat, using the function `random(MinValue, MaxValue) = RandomValue`. See the attached (proof of conceptish) `generate-random-data.pi`.

In [16]:
!cat knapsack/data.pi

instance(Items, Capacity, Values, Weights) =>
    Items = {"tv", "desktop", "laptop", "tablet", "vase", "bottle", "jacket"},
    Capacity = 23,
    Values = {500,350,230,115,180,75,125},
    Weights = {15,11,5,1,7,3,4}.